## Søke etter skjemanummer i en gitt datoperiode

In [ ]:
from google.cloud import storage
from dapla import AuthClient
import xmltodict

def loop_folder(bucket, folder, skjema):

    blobs = bucket.list_blobs(prefix=folder)
    # Loop thru the files in the folder
    for blob in blobs:

        # Jump over the folder itself
        if blob.name == folder:
            continue

        # If form file then put info in dataframe 
        blobName_list = blob.name.split('/')
        filename = blobName_list[len(blobName_list) - 1]
        incomingDate = blobName_list[0] + '/' + blobName_list[1] + '/' + blobName_list[2]

        if filename[0:5] == 'form_':
            
            # Parse the blob as a dictionary
            read_output = blob.download_as_string()
            blob_dict   = xmltodict.parse(read_output)
            
            # Fetch information
            raNummer    = blob_dict['A3_RA0689_M']['InternInfo']['raNummer']
            try:
                enhetsOrgNr = blob_dict['A3_RA0689_M']['InternInfo']['enhetsOrgNr']
            except Exception as e:
                enhetsOrgNr = '*'

            instance_list = blobName_list[len(blobName_list) - 2].split('_')
            reference     = instance_list[1]

            if raNummer == skjema:
                print('raNummer', raNummer, 'enhetsOrgNr', enhetsOrgNr, 'referanse', reference, 'ankomstdato', incomingDate)
            elif raNummer == skjema and orgnr == '':
                print('raNummer', raNummer, 'enhetsOrgNr', enhetsOrgNr, 'referanse', reference, 'ankomstdato', incomingDate)
            elif raNummer == '' and orgnr == enhetsOrgNr:
                print('raNummer', raNummer, 'enhetsOrgNr', enhetsOrgNr, 'referanse', reference, 'ankomstdato', incomingDate)

                
def search_raNummer_dateperiod(project, bucket_name, date_from, date_to, skjema):

    try:
        storage_client = storage.client.Client(credentials=AuthClient.fetch_google_credentials(), project=project,)
        bucket = storage_client.get_bucket(bucket_name)

        date_from_list      = date_from.split('/') 
        year_from           = date_from_list[0]
        month_from          = date_from_list[1]
        day_from            = date_from_list[2]
        date_to_list        = date_to.split('/') 
        year_to             = date_to_list[0]
        month_to            = date_to_list[1]
        day_to              = date_to_list[2]

        for y in range(int(year_from), int(year_to) + 1):

            if y == int(year_from):
                m_from = int(month_from)
            else:
                m_from = 1

            if y == int(year_to):
                m_to = int(month_to)
            else:
                m_to = 12


            for m in range(int(m_from), int(m_to) + 1):

                if y == int(year_from) and m == int(month_from):
                    d_from = int(day_from)
                else:
                    d_from = 1

                if y == int(year_to) and m == int(month_to):
                    d_to = int(day_to)
                else:
                    d_to = 31

                for d in range(d_from, d_to + 1):
                    folder = str(y) + '/' + str(m) + '/' + str(d) + '/'
                    loop_folder(bucket, folder, skjema)

    except Exception as e:
        print(f'Unexpected exception: {e}')
        raise e

In [ ]:
PROJECT               = "altinn-data-staging-c629"
BUCKET                = "ra0689-01-altinn-data-staging-c629-ssb-altinn"
DATE_FROM             = "2023/6/6"
DATE_TO               = "2023/7/25"
SKJEMA                = "RA-0689"

search_raNummer_dateperiod(PROJECT, BUCKET, DATE_FROM, DATE_TO, SKJEMA)